<a href="https://colab.research.google.com/github/jumbokh/encoding-BCH/blob/main/BCH_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') # 點擊網址，選擇 Google 帳號登入，然後將授權碼貼回輸入框中

Mounted at /content/gdrive


In [2]:
!ln -fs /content/gdrive/My\ Drive/Colab\ Notebooks/BCH/ /app
%cd /app

/content/gdrive/My Drive/Colab Notebooks/BCH


In [3]:
!ls # 確定目錄有訓練資料

BCH	  BCH_R_S_Input_Model.png  extract_parameters.py
BCH.csv   BCH_Train.ipynb	   Final-model.h5
BCH-logs  data			   test.h5


In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

In [5]:
data = pd.read_table("data/Training_one_to_50000.txt",sep=',',header=None)
print(data.shape)
data.head()

(50000, 75)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
0,2.236141,0.677073,1.892023,2.212047,2.173484,1.145537,3.523746,2.517586,2.123555,1.511641,2.443360,1.280934,1.639918,1.571246,2.168223,3.055222,2.858906,1.848279,-0.056266,-0.440638,1.638710,2.177900,0.426165,0.852318,2.724315,1.666961,2.745019,0.765452,2.936489,0.779298,1.724062,1.269134,1.605662,1.480742,3.165734,2.309806,1.889172,2.835951,2.273387,2.089910,3.116071,2.009234,1.173367,2.010268,1.634979,0.995538,2.377730,2.250864,1.967536,1.712788,1.519920,1.133258,1.488989,2.608448,2.005391,1.666223,2.066812,2.085648,1.620848,1.636660,1.624159,2.126042,2.140122,1,1,0,0,0,0,0,0,0,0,1,0
1,2.727983,1.644212,1.913891,1.740416,2.590680,2.130859,1.463073,3.171628,1.554147,1.765463,3.308099,-0.128945,2.482774,2.682708,0.783746,2.413238,3.105122,1.829407,1.508706,2.058221,1.893133,-0.088736,2.915489,1.788166,2.874507,3.159333,2.360416,0.990134,2.585187,1.599238,3.239353,1.375496,2.595444,1.290678,2.663287,2.321628,2.165921,0.774298,2.527509,2.741402,3.111066,0.567970,2.024100,1.341587,2.143214,2.554589,2.568119,1.967599,2.571509,1.480752,0.948844,2.460963,2.084946,2.790889,2.769237,1.664406,2.525760,1.720412,1.930328,2.572311,1.864973,2.947295,0.943815,1,1,1,0,1,0,0,1,0,1,0,1
2,2.409236,2.801631,2.057426,2.184759,2.683636,2.044105,3.262723,2.201662,2.966743,1.885213,1.384028,1.798459,2.700016,2.527087,1.012110,1.636910,-0.085207,2.001155,0.265641,2.520929,3.246987,1.737294,2.292819,2.152791,1.320763,2.282828,1.745907,1.819716,1.351256,0.271046,1.922349,1.858660,3.169719,1.412864,2.096138,1.306781,2.157087,0.627521,2.609654,-0.148885,2.650107,1.342723,2.390383,0.391234,2.884956,2.369618,1.205141,1.060666,1.686892,1.644956,1.784950,1.952461,1.551610,2.059282,3.028333,1.763497,1.428262,2.177795,1.389263,2.223563,1.935963,2.410880,2.380804,0,0,0,0,1,0,1,0,0,0,1,1
3,1.791774,1.749787,2.585813,-0.052639,2.278452,1.721495,3.100605,1.653310,2.665723,1.504311,2.248770,1.617168,2.129057,1.939706,0.518681,2.314840,2.484518,2.018851,2.611584,2.651704,1.723077,1.416195,3.063576,1.090530,-0.075914,1.531824,1.977879,0.949675,1.883779,2.723218,1.686042,3.310738,0.620059,2.416226,1.866075,1.436908,2.558286,3.027732,1.886604,1.997599,1.863093,2.328500,1.921703,3.043631,1.207121,0.584575,1.406254,1.031934,2.923467,2.832817,2.143251,1.848396,1.484815,1.875354,1.462112,1.556504,1.254478,1.782119,1.982193,1.348513,1.119968,2.453549,2.240326,1,0,1,1,1,1,1,0,1,0,1,1
4,2.095124,3.361770,2.132792,1.169481,1.952437,0.556734,1.848000,1.626273,2.178786,1.511129,2.086397,2.446340,1.861750,2.703345,1.659661,2.954063,-0.102205,1.259738,2.672633,1.937796,0.802973,2.304852,1.413590,1.907725,2.166176,1.890664,2.354610,2.771563,0.916120,1.983763,3.013018,1.881236,2.094054,2.356157,2.179067,1.203113,1.578862,2.143517,2.568300,2.227940,2.484866,3.010383,2.059089,1.269418,-0.054059,3.218174,2.843649,2.604559,2.270705,2.084287,2.809908,1.359759,2.615906,1.958774,2.398217,2.675873,1.288248,2.549243,2.750913,2.418800,1.231101,3.027114,1.427469,0,0,0,0,1,0,1,1,1,0,0,0


In [25]:
from sklearn import preprocessing   
import numpy as np    

In [28]:
#　https://aifreeblog.herokuapp.com/posts/54/data_science_203/
Min_Max_Scaler = preprocessing.MinMaxScaler( feature_range=(-1,1) ) # 設定縮放的區間上下限

In [32]:
c = np.array(data)
data_recp = c[:,:63]
MinMax_Data = Min_Max_Scaler.fit_transform( data_recp ) # Data 為原始資料

In [33]:
print(MinMax_Data.shape)
print(MinMax_Data[:5])

(50000, 63)
[[ 0.16526226 -0.38988075 -0.02269398  0.16133774  0.14035665 -0.18627233
   0.48127444  0.26965293  0.05198578 -0.1236961   0.19623933 -0.0816841
  -0.08163587 -0.05482516  0.09061733  0.36265302  0.29256801  0.03144075
  -0.48678055 -0.70714107 -0.04431095  0.14307799 -0.38513189 -0.35302213
   0.33823341  0.04047587  0.31357988 -0.3435214   0.33521773 -0.32394972
  -0.13782901 -0.21811168 -0.08205822 -0.12338132  0.42959645  0.21186972
   0.00567064  0.34281986  0.09391528  0.09841882  0.4096713   0.07219989
  -0.17104073  0.06078598 -0.03253457 -0.24751141  0.10577498  0.14099224
   0.09357883 -0.02285654 -0.08320669 -0.16946341 -0.13845678  0.29875817
   0.08114705 -0.02890881  0.06404679  0.10598091 -0.13500534 -0.04734413
  -0.09816704  0.11109946  0.04287932]
 [ 0.32877798 -0.10020427 -0.0155947   0.00954863  0.27957178  0.13046238
  -0.1312923   0.48681309 -0.12357932 -0.04081672  0.47005905 -0.52517876
   0.19627347  0.29465622 -0.36664966  0.17343864  0.37194958 

In [34]:
np.array(MinMax_Data).mean()

0.04495322885969454

In [42]:
b = np.array(data)
data_syndrom = b[:,63:len(b[0])]
print(data_syndrom.shape)
print(data_syndrom[:5])

(50000, 12)
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1.]
 [0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0.]]


# Data 最大值 最小值

In [43]:
print(MinMax_Data.max())
print(MinMax_Data.min())

1.0000000000000002
-1.0


In [ ]:
pd.DataFrame(data_syndrom).dtypes

0     float64
1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
dtype: object

In [44]:
datasets = pd.read_table("data/Validation_one_to_50000.txt",sep=',',header=None)
print(datasets.shape)
print(datasets.head())

(50000, 63)
   0   1   2   3   4   5   6   7   8   ...  54  55  56  57  58  59  60  61  62
0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0
1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0
2   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0
3   0   0   0   1   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0
4   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0

[5 rows x 63 columns]


In [ ]:
data_num = datasets.shape[0]
# 取得一筆與data數量相同的亂數索引，主要目的是用於打散資料
indexes = np.random.permutation(data_num)
# 並將亂數索引值分為Train、val和test分為，這裡的劃分比例為6:2:2
train_indexes = indexes[:int(data_num *0.6)]
val_indexes = indexes[int(data_num *0.6):int(data_num *0.8)]
test_indexes = indexes[int(data_num *0.8):]
# 透過索引值從data取出訓練資料、驗證資料和測試資料
x_train = datasets.loc[train_indexes]
x_val = datasets.loc[val_indexes]
x_test = datasets.loc[test_indexes]

In [ ]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(30000, 63)
(10000, 63)
(10000, 63)


In [ ]:
syndrom = pd.DataFrame(data_syndrom)
x_train1 = syndrom.loc[train_indexes]
x_val1 = syndrom.loc[val_indexes]
x_test1 = syndrom.loc[test_indexes]

In [ ]:
print(x_train1.shape)
print(x_val1.shape)
print(x_test1.shape)

(30000, 12)
(10000, 12)
(10000, 12)
